In [1]:
from dataclasses import dataclass
from typing import Tuple

from numpy import uint64 as u64
import memprocfs

In [15]:
def show(bytes):
    print(vmm.hex(bytes))

In [2]:
@dataclass
class CS2Offsets:
    cs2_entity_list: u64
    cs2_view_matrix: u64
    cs2_client_state: u64
    cs2_local_player: u64
    cs2_view_angle: u64
    cs2_map: u64
    cs2_player_info: u64
    cs2_life_state: u64
    cs2_ang_eye_angle: u64
    cs2_dormant: u64
    cs2_team: u64
    cs2_health: u64
    cs2_origin: u64

In [3]:
@dataclass
class EntityInfo:
    team: int
    health: int
    lifestate: int
    origin: Tuple[float, float, float]
    origin_forward: Tuple[float, float, float]
    view_angle: Tuple[float, float]
    local: int
    valid: int
    name: str

In [4]:
# Initialize base object vmm from dump file
vmm = memprocfs.Vmm(['-device', 'fpga'], )

In [5]:
process = vmm.process('cs2.exe')

In [6]:
client = process.module('client.dll')

In [36]:
local_player = process.memory.read(client.base+0x171C6E8, 0x8, memprocfs.FLAG_NOCACHE)

In [63]:
t = int.from_bytes(local_player, 'little')

In [69]:
show(process.memory.read(t, 0xFF, memprocfs.FLAG_NOCACHE))

0000    c8 e4 48 d1 fe 7f 00 00  00 00 00 00 00 00 00 00   ..H.. ..........
0010    18 3b c9 ae 5b 02 00 00  00 00 00 00 00 00 00 00   .;..[...........
0020    00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00   ................
0030    00 00 bd 5a 5b 02 00 00  20 b2 44 d1 fe 7f 00 00   ...Z[... .D.. ..
0040    00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00   ................
0050    00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00   ................
0060    00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00   ................
0070    00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00   ................
0080    00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00   ................
0090    00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00   ................
00a0    00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00   ................
00b0    00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00   ................
00c0    00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00   ................
00d0    00 0

In [70]:
show(process.memory.read(t, 0xFF, memprocfs.FLAG_NOCACHE))

0000    c8 e4 48 d1 fe 7f 00 00  00 00 00 00 00 00 00 00   ..H.. ..........
0010    18 3b c9 ae 5b 02 00 00  00 00 00 00 00 00 00 00   .;..[...........
0020    00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00   ................
0030    00 00 bd 5a 5b 02 00 00  20 b2 44 d1 fe 7f 00 00   ...Z[... .D.. ..
0040    00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00   ................
0050    00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00   ................
0060    00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00   ................
0070    00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00   ................
0080    00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00   ................
0090    00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00   ................
00a0    00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00   ................
00b0    00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00   ................
00c0    00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00   ................
00d0    00 0

In [42]:
rules = {
    'global':' rule Globals { strings: $pat = { 48 89 15 ?? ?? ?? ?? 48 8D 05 ?? ?? ?? ?? 48 85 D2 } condition: $pat } ',
    'local_player':' rule LocalPlayer { strings: $pat = { 48 83 3D ? ? ? ? ? 0F 95 } condition: $pat } ',
    'entity_list':' rule EntityList { strings: $pat = { 4C 8B 0D ? ? ? ? 48 89 5C 24 ? 8B } condition: $pat } ',
}

In [46]:
rules = {
    'global':' rule Globals { strings: $pat = { 48 89 15 ?? ?? ?? ?? 48 8D 05 ?? ?? ?? ?? 48 85 D2 } condition: $pat } ',
    'local_player':' rule LocalPlayer { strings: $pat = { 48 83 3D ?? ?? ?? ?? ?? 0F 95 } condition: $pat } ',
}

In [ ]:
offset = 0x03
inst_length = 0x07

In [47]:
yara = process.search_yara(rules['local_player'], client.base)

In [48]:
yara.max_results = 10

In [49]:
yara.result()

[]